In [22]:
import requests
import polars as pl
from io import BytesIO
import os


In [25]:
# Base URL of your FastAPI app
base_url = 'http://localhost:8082'

token="sVcvt0D3fSqNOEeO9KEH"


In [28]:
headers = {
    'Authorization': f'Bearer {token}',
}

# 2. Create a sample Parquet file (if you don't have one)

# Create a Polars DataFrame
df = pl.DataFrame({
    'column1': [10, 20, 30],
    'column2': ['x', 'y', 'z'],
})

# Save DataFrame to a Parquet file in memory
parquet_buffer = BytesIO()
df.write_parquet(parquet_buffer)
parquet_buffer.seek(0)  # Reset buffer position

files = {
    'file': ('sample.parquet', parquet_buffer, 'application/octet-stream'),
}

upload_response = requests.post(
    f'{base_url}/upload',
    files=files,
    headers=headers,
)

print('Upload Response:')
print(upload_response.status_code, upload_response.json())

Upload Response:
500 {'detail': "name 'PARQUET_DIR' is not defined"}


In [21]:
files = {
    'file': ('sample.parquet', parquet_buffer, 'application/octet-stream'),
}

upload_response = requests.post(
    f'{base_url}/upload',
    files=files,
    headers=headers,
)

print('Upload Response:')
print(upload_response.status_code, upload_response.json())


Upload Response:
422 {'detail': [{'type': 'missing', 'loc': ['query', 'authorization_header'], 'msg': 'Field required', 'input': None}]}


In [ ]:

# 4. List all Parquet files and folders

list_response = requests.get(
    f'{base_url}/files',
    headers=headers,
)

print('\nFiles List Response:')
print(list_response.status_code)
print(list_response.json())

In [ ]:

# 5. Download the uploaded Parquet file

# Extract the filename from the upload response
uploaded_filename = upload_response.json().get('filename')

if uploaded_filename:
    download_response = requests.get(
        f'{base_url}/download/{uploaded_filename}',
        headers=headers,
    )
    
    if download_response.status_code == 200:
        # Read the content into a BytesIO buffer
        downloaded_buffer = BytesIO(download_response.content)
        
        # Read the Parquet file into a DataFrame
        downloaded_df = pl.read_parquet(downloaded_buffer)
        print('\nDownloaded DataFrame:')
        print(downloaded_df)
    else:
        print(f'Error downloading file: {download_response.status_code} {download_response.text}')
else:
    print('No filename received from upload response.')
